## Week 3 : SIR Process

Imagine a disease.
Nodes are in 3 states, S/I/R
When a node is I, they will remain I for 1 week and then be R.
When a node is I, they will cause their adjacent nodes to be I with probability $\lambda$, independently.
A person who is R will no longer partake in any dynamic process.

To start the outbreak, define an initial condition. 
Almost all the nodes are S, a small fraction of nodes chosen uniformly at random, begin as I. 

Terminal state is when we only have nodes of S/R and no I left. 

Assumptions:

We assume no one is added to S group, the only way a node leaves the S group is by becoming I.
Assumed a fixed fraction k of the infected group will recover during any given day. In this case since duration of infection is 3 days then $\frac{1}{7}$.

Differential Relationships : https://maa.org/press/periodicals/loci/joma/the-sir-model-for-spread-of-disease-the-differential-equation-model

Implementing in Python : https://pythonhosted.org/epydemic/sir.html#:~:text=The%20Susceptible%2DInfected%2DRemoved%20compartmented,Kermack%20and%20McKendrick%20in%201927.

# Q1

Generating model with nodes $n$ and mean $k$ : 

Motivation for the Configuration Model over the Random Model

The configuration model is a model in which the degrees of vertices are fixed beforehand. Such a model is more flexible than the generalized random graph. For example, the generalized random graph always has a positive proportion of vertices of degree 0, 1, 2, etc. (see Exercises 6.15–6.16). In some real-world networks, however, it is natural to investigate graphs where every vertex has at least one or two neighbors.

For various $\lambda$ values, run until only S/R state are left. 

In [1]:
import numpy as np
import scipy.stats as scistats

In [12]:
class Network():

    def __init__(self, num_nodes) -> None:
        self.state =['S' for _ in range(num_nodes)]
        self.adj = {i:set() for i in range(num_nodes)}
        self.num_edge = 0 
        self.susceptible = set([i for i in range(num_nodes)])
        self.infectious = set()
        self.recovered = set()

    def add_edge(self,i,j):
        self.adj[i].add(j)
        self.adj[j].add(i)
        self.num_edge+=1

    def edge_list(self):
        return [(i,j) for i in self.adj for j in self.adj[i] if i<j]
    
    def infect(self,i):
        self.state[i] = 'I'
        self.infectious.add(i)
        self.susceptible.remove(i)

    def remove(self,i):
        self.state[i] = 'R'
        self.recovered.add(i)
        self.infectious.remove(i)

    

In [21]:
class Poisson_Network(Network):

    def __init__(self, num_nodes, mean, p_infected, p_infect) -> None:
        super().__init__(num_nodes)

        self.p_infect = p_infect

        k = np.random.poisson(lam=mean,size=num_nodes)

        S = np.array([i for i in range(num_nodes) for _ in range(k[i])])
        S = np.random.permutation(S)
        
        if len(S)%2:
            S = S[:-1]

        S = S.reshape(-1,2)

        for i,j in S:
            if i!=j and i not in self.adj[j]:
                self.add_edge(i,j)

        self.infected = np.random.randint(0,num_nodes,size = int(p_infected*num_nodes))

        for i in self.infected:
            if self.state[i]=='S':
                self.infect(i)

    def run(self):
        '''Runs simulation for a cycle, returns SIR value at end of cycle'''
        curr = list(self.infectious)
        for i in curr:
            for adj in self.adj[i]:
                if self.state[adj] == 'S':
                    outcome = np.random.binomial(1,self.p_infect)
                    if outcome: 
                        self.infect(adj)
                        
            self.remove(i)

        return len(self.susceptible), len(self.infectious), len(self.recovered)   


    def run_to_extinction(self):
        '''Runs simulation until only S/R state is left'''

        while self.infectious:
            i = self.infectious.pop()
            for adj in self.adj[i]:
                if self.state[adj] == 'S':
                    outcome = np.random.binomial(1,self.p_infect)
                    if outcome: 
                        self.infect(adj)
            self.recovered.add(i)

        return len(self.susceptible), len(self.infectious), len(self.recovered)   

In [24]:
n = 10000
mean = 20 
p_infectious_array = np.lin

9294 606 100
6396 2898 706
1178 5218 3604
45 1133 8822
21 24 9955
21 0 9979
